<a href="https://colab.research.google.com/github/Jitwisut/web-deepface/blob/main/AI-deepface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

# Remove existing files in the mount point directory
if os.path.exists('/content/drive') and os.listdir('/content/drive'):
    !rm -rf /content/drive/*

drive.mount('/content/drive', force_remount=True)

rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/Shareddrives': Operation canceled
Mounted at /content/drive


In [ ]:
!pip install pyngrok
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=86a5ffcd2e321d8ca211ad40621d5fb93199957ab079d19004225617dc106ffa
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import base64
import os
import json
import pickle
import numpy as np
from flask import Flask, request, render_template_string, jsonify, redirect, url_for
from pyngrok import ngrok
from PIL import Image
import io
import csv
import pandas as pd  # เพิ่ม pandas เพื่อจัดการ CSV ได้ง่ายขึ้น

# --- ส่วนนำเข้า DeepFace ---
from deepface import DeepFace
print("✅ DeepFace พร้อมใช้งาน")

# --- ส่วนตั้งค่า ---
app = Flask(__name__)
SAVE_DIR = "/content/drive/My Drive/UserImages"
ENCODINGS_DIR = "/content/drive/My Drive/FaceEncodings_DeepFace"
USER_DATA_CSV = "/content/drive/My Drive/user_data.csv"  # ไฟล์ CSV สำหรับเก็บข้อมูลผู้ใช้
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(ENCODINGS_DIR, exist_ok=True)

# ตรวจสอบและสร้างไฟล์ CSV ถ้ายังไม่มี
if not os.path.exists(USER_DATA_CSV):
    with open(USER_DATA_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["username", "fullname", "phone", "first_image_path"])

# ======================================================================
#                               HTML TEMPLATES
# ======================================================================

HOME_PAGE = '''
<!DOCTYPE html>
<html>
<head>
  <title>ระบบจดจำใบหน้า AI</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    /* ---------- GENERAL ---------- */
    *{margin:0;padding:0;box-sizing:border-box}
    body{
      font-family:'Segoe UI',Tahoma,Geneva,Verdana,sans-serif;
      background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);
      min-height:100vh;display:flex;align-items:center;justify-content:center
    }
    .container{
      background:rgba(255,255,255,.95);backdrop-filter:blur(10px);
      border-radius:20px;padding:40px;box-shadow:0 20px 40px rgba(0,0,0,.1);
      text-align:center;max-width:400px;width:90%
    }
    .logo{font-size:60px;margin-bottom:20px}
    h1{color:#333;margin-bottom:10px;font-size:28px;font-weight:600}
    .subtitle{color:#666;margin-bottom:40px;font-size:16px}
    .btn{
      display:block;width:100%;padding:15px 30px;margin:15px 0;
      background:linear-gradient(45deg,#667eea,#764ba2);color:white;
      text-decoration:none;border-radius:12px;font-size:18px;font-weight:500;
      transition:.3s;border:none;cursor:pointer
    }
    .btn:hover{transform:translateY(-2px);box-shadow:0 10px 25px rgba(102,126,234,.3)}
    .btn.secondary{background:linear-gradient(45deg,#28a745,#20c997)}
    .btn.secondary:hover{box-shadow:0 10px 25px rgba(40,167,69,.3)}
    .features{margin-top:30px;text-align:left}
    .feature{display:flex;align-items:center;margin:10px 0;color:#555}
    .feature-icon{margin-right:10px;font-size:18px}
    .ai-badge{
      background:linear-gradient(45deg,#ff6b6b,#ff8e8e);color:white;
      padding:5px 12px;border-radius:20px;font-size:12px;margin-left:10px;font-weight:bold
    }
  </style>
</head>
<body>
  <div class="container">
    <div class="logo">🤖</div>
    <h1>ระบบจดจำใบหน้า AI <span class="ai-badge">DeepFace + Flex</span></h1>
    <p class="subtitle">เข้าสู่ระบบด้วยใบหน้าของคุณ</p>
    <a href="/login"  class="btn">🔍 เข้าสู่ระบบด้วย AI</a>
    <a href="/signup" class="btn secondary">📝 ลงทะเบียนใหม่</a>
    <div class="features">
      <div class="feature"><span class="feature-icon">⚡</span><span>เข้าสู่ระบบรวดเร็ว</span></div>
      <div class="feature"><span class="feature-icon">🧠</span><span>AI จดจำใบหน้าแม่นยำ (แม้ภาพไม่สมบูรณ์)</span><span class="ai-badge">Flex Mode</span></div>
      <div class="feature"><span class="feature-icon">🔒</span><span>ปลอดภัยสูง</span></div>
    </div>
  </div>
</body>
</html>
'''
SIGNUP_PAGE = '''
<!DOCTYPE html>
<html>
<head>
  <title>ลงทะเบียนผู้ใช้ใหม่</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    * {
      margin: 0;
      padding: 0;
      box-sizing: border-box;
    }
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      background: linear-gradient(135deg, #28a745 0%, #20c997 100%);
      min-height: 100vh;
      display: flex;
      align-items: center;
      justify-content: center;
      padding: 20px;
    }
    .container {
      background: rgba(255, 255, 255, 0.95);
      backdrop-filter: blur(10px);
      border-radius: 20px;
      padding: 40px;
      box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
      text-align: center;
      max-width: 500px;
      width: 100%;
    }
    .instructions {
      background: #f1f8e9;
      border-left: 5px solid #28a745;
      color: #2c5f2d;
      text-align: left;
      margin-bottom: 25px;
      font-size: 15px;
      padding: 16px 20px 16px 15px;
      border-radius: 12px;
      line-height: 1.7;
    }
    .back-btn {
      position: absolute;
      top: 20px;
      left: 20px;
      background: rgba(255, 255, 255, 0.2);
      color: white;
      border: none;
      padding: 10px 15px;
      border-radius: 10px;
      cursor: pointer;
      font-size: 16px;
      backdrop-filter: blur(10px);
    }
    h2 {
      color: #333;
      margin-bottom: 10px;
      font-size: 24px;
    }
    .subtitle {
      color: #666;
      margin-bottom: 30px;
    }
    #video {
      width: 100%;
      max-width: 400px;
      height: 300px;
      border-radius: 15px;
      border: 3px solid #28a745;
      margin-bottom: 20px;
      object-fit: cover;
      transform: scaleX(-1);
    }
    .form-group {
      margin-bottom: 20px;
      text-align: left;
    }
    .form-group label {
      display: block;
      margin-bottom: 8px;
      color: #555;
      font-weight: 500;
    }
    .form-group input[type="text"] {
      width: 100%;
      padding: 12px 15px;
      border: 1px solid #ddd;
      border-radius: 8px;
      font-size: 16px;
    }
    .btn {
      background: linear-gradient(45deg, #28a745, #20c997);
      color: white;
      font-size: 18px;
      padding: 15px 30px;
      border: none;
      border-radius: 12px;
      cursor: pointer;
      transition: all 0.3s ease;
      margin: 10px 0;
    }
    .btn:hover {
      transform: translateY(-2px);
      box-shadow: 0 10px 25px rgba(40, 167, 69, 0.3);
    }
    .btn:disabled {
      opacity: 0.6;
      cursor: not-allowed;
      transform: none;
    }
    .capture-buttons {
      display: grid;
      grid-template-columns: repeat(5, 1fr);
      gap: 8px;
      margin-bottom: 20px;
    }
    .capture-buttons button {
      padding: 10px 8px;
      border-radius: 8px;
      border: none;
      background-color: #007bff;
      color: white;
      cursor: pointer;
      transition: all 0.2s;
      font-size: 12px;
      position: relative;
    }
    .capture-buttons button:hover {
      background-color: #0056b3;
      transform: scale(1.05);
    }
    .capture-buttons button.captured {
      background-color: #28a745;
    }
    .capture-buttons button.captured::after {
      content: "✓";
      position: absolute;
      top: 2px;
      right: 2px;
      font-size: 10px;
      background: white;
      color: #28a745;
      border-radius: 50%;
      width: 16px;
      height: 16px;
      display: flex;
      align-items: center;
      justify-content: center;
    }
    #currentInstruction {
      background: #e3f2fd;
      border: 2px solid #2196f3;
      border-radius: 10px;
      padding: 15px;
      margin: 15px 0;
      font-size: 16px;
      color: #1565c0;
      font-weight: 500;
      min-height: 60px;
      display: flex;
      align-items: center;
      justify-content: center;
      text-align: center;
    }
    #captureStatus {
      margin-top: 10px;
      font-size: 14px;
      color: #777;
      min-height: 20px;
    }
    #overallStatus {
      margin-top: 20px;
      font-size: 16px;
      color: #444;
      padding: 15px;
      border-radius: 10px;
      min-height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
    }
    .success {
      background: #d4edda;
      color: #155724;
      border: 1px solid #c3e6cb;
    }
    .error {
      background: #f8d7da;
      color: #721c24;
      border: 1px solid #f5c6cb;
    }
    .loading {
      background: #d1ecf1;
      color: #0c5460;
      border: 1px solid #bee5eb;
    }
    @media (max-width: 480px) {
      .capture-buttons {
        grid-template-columns: repeat(3, 1fr);
      }
    }
  </style>
</head>
<body>
  <button class="back-btn" onclick="window.location.href='/'">← กลับ</button>
  <div class="container">
    <h2>📝 ลงทะเบียนผู้ใช้ใหม่</h2>
    <p class="subtitle">กรอกข้อมูลและถ่ายภาพใบหน้า 15 มุม</p>

    <div class="form-group">
      <label for="username">ชื่อผู้ใช้ (Username):</label>
      <input type="text" id="username" placeholder="เช่น testuser (ภาษาอังกฤษเท่านั้น)" required>
    </div>
    <div class="form-group">
      <label for="fullname">ชื่อ-นามสกุลจริง:</label>
      <input type="text" id="fullname" placeholder="เช่น สมชาย ใจดี" required>
    </div>
    <div class="form-group">
      <label for="phone">เบอร์โทรศัพท์:</label>
      <input type="text" id="phone" placeholder="เช่น 08XXXXXXXX" required>
    </div>

    <video id="video" autoplay playsinline muted></video><br>

    <div id="currentInstruction">
      กรุณากรอกข้อมูลและกดปุ่ม "มุม 1" เพื่อเริ่มถ่ายภาพ
    </div>

    <div class="capture-buttons">
      <button id="btn1" onclick="captureImage(1)">มุม 1</button>
      <button id="btn2" onclick="captureImage(2)">มุม 2</button>
      <button id="btn3" onclick="captureImage(3)">มุม 3</button>
      <button id="btn4" onclick="captureImage(4)">มุม 4</button>
      <button id="btn5" onclick="captureImage(5)">มุม 5</button>
      <button id="btn6" onclick="captureImage(6)">มุม 6</button>
      <button id="btn7" onclick="captureImage(7)">มุม 7</button>
      <button id="btn8" onclick="captureImage(8)">มุม 8</button>
      <button id="btn9" onclick="captureImage(9)">มุม 9</button>
      <button id="btn10" onclick="captureImage(10)">มุม 10</button>
      <button id="btn11" onclick="captureImage(11)">มุม 11</button>
      <button id="btn12" onclick="captureImage(12)">มุม 12</button>
      <button id="btn13" onclick="captureImage(13)">มุม 13</button>
      <button id="btn14" onclick="captureImage(14)">มุม 14</button>
      <button id="btn15" onclick="captureImage(15)">มุม 15</button>
    </div>
    <div id="captureStatus"></div>
    <button id="trainBtn" class="btn" onclick="trainUser()" disabled>🚀 เริ่มเทรน AI</button>
    <div id="overallStatus">กรอกข้อมูลและถ่ายภาพทั้ง 15 มุม เพื่อเริ่มเทรน AI</div>
  </div>

  <script>
    const video = document.getElementById('video');
    const captureStatusDiv = document.getElementById('captureStatus');
    const overallStatusDiv = document.getElementById('overallStatus');
    const currentInstructionDiv = document.getElementById('currentInstruction');
    const trainBtn = document.getElementById('trainBtn');
    const usernameInput = document.getElementById('username');
    const fullnameInput = document.getElementById('fullname');
    const phoneInput = document.getElementById('phone');

    const capturedAngles = new Set();
    const totalAngles = 15;

    // คำแนะนำสำหรับแต่ละมุม
    const angleInstructions = {
      1: "📸 มุมที่ 1: ถ่ายหน้าตรง - มองตรงเข้ากล้อง ใบหน้าตั้งตรง",
      2: "📸 มุมที่ 2: หันหน้าไปทางขวาเล็กน้อย (15 องศา) - ยังมองเห็นตาทั้งสองข้าง",
      3: "📸 มุมที่ 3: หันหน้าไปทางซ้ายเล็กน้อย (15 องศา) - ยังมองเห็นตาทั้งสองข้าง",
      4: "📸 มุมที่ 4: หันหน้าไปทางขวามากขึ้น (30 องศา) - เห็นข้างหน้าชัดเจน",
      5: "📸 มุมที่ 5: หันหน้าไปทางซ้ายมากขึ้น (30 องศา) - เห็นข้างหน้าชัดเจน",
      6: "📸 มุมที่ 6: ก้มหน้าลงเล็กน้อย - คางลงมาเล็กน้อย ยังมองเห็นตา",
      7: "📸 มุมที่ 7: เงยหน้าขึ้นเล็กน้อย - คางยกขึ้นเล็กน้อย",
      8: "📸 มุมที่ 8: เอียงศีรษะไปทางขวา - หูขวาลงมาใกล้ไหล่ขวา",
      9: "📸 มุมที่ 9: เอียงศีรษะไปทางซ้าย - หูซ้ายลงมาใกล้ไหล่ซ้าย",
      10: "📸 มุมที่ 10: หันขวา + ก้มหน้า - รวมการหันขวาและก้มหน้าเล็กน้อย",
      11: "📸 มุมที่ 11: หันซ้าย + ก้มหน้า - รวมการหันซ้ายและก้มหน้าเล็กน้อย",
      12: "📸 มุมที่ 12: หันขวา + เงยหน้า - รวมการหันขวาและเงยหน้าเล็กน้อย",
      13: "📸 มุมที่ 13: หันซ้าย + เงยหน้า - รวมการหันซ้ายและเงยหน้าเล็กน้อย",
      14: "📸 มุมที่ 14: หันขวามาก (45 องศา) - เห็นเพียงข้างหน้าเท่านั้น",
      15: "📸 มุมที่ 15: หันซ้ายมาก (45 องศา) - เห็นเพียงข้างหน้าเท่านั้น"
    };

    navigator.mediaDevices.getUserMedia({ video: { facingMode: "user" } })
      .then(stream => {
        video.srcObject = stream;
        overallStatusDiv.textContent = "กล้องพร้อมใช้งาน กรุณากรอกข้อมูลและถ่ายภาพ";
      })
      .catch(err => {
        overallStatusDiv.textContent = "❌ ไม่สามารถเปิดกล้องได้: " + err.message;
        overallStatusDiv.className = "error";
      });

    function updateCurrentInstruction(angle) {
      if (angleInstructions[angle]) {
        currentInstructionDiv.textContent = angleInstructions[angle];
        currentInstructionDiv.style.background = "#fff3cd";
        currentInstructionDiv.style.borderColor = "#ffc107";
        currentInstructionDiv.style.color = "#856404";
      }
    }

    function getNextAngleInstruction() {
      for (let i = 1; i <= totalAngles; i++) {
        if (!capturedAngles.has(i)) {
          return `ถัดไป: กดปุ่ม "มุม ${i}" - ${angleInstructions[i]}`;
        }
      }
      return "🎉 ถ่ายภาพครบทุกมุมแล้ว! พร้อมเทรน AI";
    }

    async function captureImage(angle) {
      const username = usernameInput.value.trim();
      if (!username) {
        alert("กรุณาใส่ชื่อผู้ใช้ก่อนถ่ายภาพ!");
        return;
      }

      // แสดงคำแนะนำสำหรับมุมที่จะถ่าย
      updateCurrentInstruction(angle);

      if (capturedAngles.has(angle)) {
        if (!confirm(`คุณได้ถ่ายภาพมุมที่ ${angle} ไปแล้ว ต้องการถ่ายใหม่หรือไม่?`)) {
          return;
        }
      }

      captureStatusDiv.textContent = `กำลังถ่ายภาพมุมที่ ${angle}...`;

      try {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const ctx = canvas.getContext('2d');
        ctx.scale(-1, 1);
        ctx.drawImage(video, -canvas.width, 0);
        const imageData = canvas.toDataURL('image/jpeg').split(',')[1];

        const res = await fetch('/save_image', {
          method: 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify({ username: username, angle: angle, image: imageData })
        });
        const result = await res.json();

        if (result.success) {
          captureStatusDiv.textContent = `✅ ${result.message}`;
          capturedAngles.add(angle);

          // เปลี่ยนสีปุ่มเป็นเขียวเมื่อถ่ายเสร็จ
          const button = document.getElementById(`btn${angle}`);
          button.classList.add('captured');

          // แสดงคำแนะนำสำหรับมุมถัดไป
          if (capturedAngles.size < totalAngles) {
            currentInstructionDiv.textContent = getNextAngleInstruction();
            currentInstructionDiv.style.background = "#e3f2fd";
            currentInstructionDiv.style.borderColor = "#2196f3";
            currentInstructionDiv.style.color = "#1565c0";
          } else {
            currentInstructionDiv.textContent = "🎉 ถ่ายภาพครบทุกมุมแล้ว! พร้อมเทรน AI";
            currentInstructionDiv.style.background = "#d4edda";
            currentInstructionDiv.style.borderColor = "#28a745";
            currentInstructionDiv.style.color = "#155724";
          }

          updateTrainButtonStatus();
        } else {
          captureStatusDiv.textContent = `❌ ${result.message}`;
          captureStatusDiv.className = "error";
        }
      } catch (error) {
        captureStatusDiv.textContent = `❌ เกิดข้อผิดพลาดในการถ่ายภาพ: ${error.message}`;
        captureStatusDiv.className = "error";
      }
    }

    function updateTrainButtonStatus() {
      const username = usernameInput.value.trim();
      const fullname = fullnameInput.value.trim();
      const phone = phoneInput.value.trim();

      if (username && fullname && phone && capturedAngles.size === totalAngles) {
        trainBtn.disabled = false;
        overallStatusDiv.textContent = "✅ ถ่ายภาพครบทุกมุมแล้ว พร้อมเทรน AI!";
        overallStatusDiv.className = "success";
      } else {
        trainBtn.disabled = true;
        overallStatusDiv.textContent = `กรอกข้อมูลและถ่ายภาพครบ ${capturedAngles.size}/${totalAngles} มุม เพื่อเริ่มเทรน AI`;
        overallStatusDiv.className = "";
      }
    }

    usernameInput.addEventListener('input', updateTrainButtonStatus);
    fullnameInput.addEventListener('input', updateTrainButtonStatus);
    phoneInput.addEventListener('input', updateTrainButtonStatus);

    async function trainUser() {
      trainBtn.disabled = true;
      overallStatusDiv.textContent = "🚀 AI กำลังประมวลผลและเรียนรู้ใบหน้าของคุณ (อาจใช้เวลา 1-2 นาที)...";
      overallStatusDiv.className = "loading";

      const username = usernameInput.value.trim();
      const fullname = fullnameInput.value.trim();
      const phone = phoneInput.value.trim();

      try {
        const res = await fetch('/train_user', {
          method: 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify({
            username: username,
            fullname: fullname,
            phone: phone
          })
        });
        const result = await res.json();

        if (result.success) {
          overallStatusDiv.textContent = `🎉 ${result.message} คุณ ${fullname} ยินดีต้อนรับ!`;
          overallStatusDiv.className = "success";
          alert(`ลงทะเบียนสำเร็จ! ชื่อผู้ใช้: ${result.username}`);
          window.location.href = '/'; // กลับไปหน้าหลัก
        } else {
          overallStatusDiv.textContent = `❌ ${result.message}`;
          overallStatusDiv.className = "error";
        }
      } catch (error) {
        overallStatusDiv.textContent = `❌ เกิดข้อผิดพลาดในการเทรน AI: ${error.message}`;
        overallStatusDiv.className = "error";
      }
      trainBtn.disabled = false;
    }
  </script>
</body>
</html>
'''

# ----------------------------------------------------------------------
# LOGIN_PAGE (เพิ่ม CSS + JS สำหรับ Popup และแก้ redirect)
# ----------------------------------------------------------------------
LOGIN_PAGE = '''
<!DOCTYPE html>
<html>
<head>
  <title>เข้าสู่ระบบด้วย AI - ระบบจดจำใบหน้า</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    /* ---------- GENERAL ---------- */
    *{margin:0;padding:0;box-sizing:border-box}
    body{
      font-family:'Segoe UI',Tahoma,Geneva,Verdana,sans-serif;
      background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);
      min-height:100vh;display:flex;align-items:center;justify-content:center;padding:20px
    }
    .container{
      background:rgba(255,255,255,.95);backdrop-filter:blur(10px);
      border-radius:20px;padding:40px;box-shadow:0 20px 40px rgba(0,0,0,.1);
      text-align:center;max-width:450px;width:100%
    }
    .back-btn{
      position:absolute;top:20px;left:20px;background:rgba(255,255,255,.2);
      color:white;border:none;padding:10px 15px;border-radius:10px;cursor:pointer;
      font-size:16px;backdrop-filter:blur(10px)
    }
    h2{color:#333;margin-bottom:10px;font-size:24px}
    .subtitle{color:#666;margin-bottom:30px}
    #video{
      width:100%;max-width:350px;height:260px;border-radius:15px;border:3px solid #667eea;
      margin-bottom:20px;object-fit:cover;transform:scaleX(-1)
    }
    .btn{
      background:linear-gradient(45deg,#667eea,#764ba2);color:white;font-size:18px;
      padding:15px 30px;border:none;border-radius:12px;cursor:pointer;transition:.3s;margin:10px
    }
    .btn:hover{transform:translateY(-2px);box-shadow:0 10px 25px rgba(102,126,234,.3)}
    .btn:disabled{opacity:.6;cursor:not-allowed;transform:none}
    #status{
      margin-top:20px;font-size:16px;color:#444;padding:15px;border-radius:10px;
      min-height:50px;display:flex;align-items:center;justify-content:center
    }
    .success{background:#d4edda;color:#155724;border:1px solid #c3e6cb}
    .error  {background:#f8d7da;color:#721c24;border:1px solid #f5c6cb}
    .loading{background:#d1ecf1;color:#0c5460;border:1px solid #bee5eb}
    .ai-badge{
      background:linear-gradient(45deg,#ff6b6b,#ff8e8e);color:white;
      padding:3px 8px;border-radius:12px;font-size:10px;margin-left:5px;font-weight:bold
    }
    /* ---------- POP-UP ---------- */
    .success-popup-overlay{
      position:fixed;inset:0;background:rgba(0,0,0,.7);
      display:flex;justify-content:center;align-items:center;z-index:9999;
      opacity:0;transition:.3s
    }
    .success-popup-overlay.show{opacity:1}
    .success-popup-content{
      background:#fff;padding:30px;border-radius:15px;max-width:380px;width:90%;text-align:center;
      box-shadow:0 10px 30px rgba(0,0,0,.3);transform:scale(.8);transition:.3s
    }
    .success-popup-overlay.show .success-popup-content{transform:scale(1)}
    .success-popup-content h2{margin:15px 0;color:#2e7d32}
    .popup-btn{
      margin-top:20px;padding:12px 28px;border:0;border-radius:25px;
      background:linear-gradient(135deg,#4CAF50,#2e7d32);
      color:#fff;font-size:16px;cursor:pointer;transition:transform .2s
    }
    .popup-btn:hover{transform:translateY(-2px);box-shadow:0 5px 15px rgba(76,175,80,.4)}
  </style>
</head>
<body>
  <button class="back-btn" onclick="window.location.href='/'">← กลับ</button>
  <div class="container">
    <h2>🤖 เข้าสู่ระบบด้วย AI <span class="ai-badge">Flex Mode</span></h2>
    <p class="subtitle">มองไปที่กล้องเพื่อให้ AI ยืนยันตัวตน</p>
    <video id="video" autoplay playsinline muted></video><br>
    <button id="loginBtn" class="btn" onclick="login()">🧠 ยืนยันด้วย AI</button>
    <div id="status">กดปุ่มเพื่อให้ AI ตรวจสอบใบหน้าของคุณ</div>
  </div>
  <script>
    const video      = document.getElementById('video');
    const statusDiv  = document.getElementById('status');
    const loginBtn   = document.getElementById('loginBtn');

    // -------- Camera --------
    navigator.mediaDevices.getUserMedia({video:{facingMode:"user"}})
      .then(stream=>{
        video.srcObject = stream;
        statusDiv.textContent = "กล้องพร้อมใช้งาน กดปุ่มเพื่อให้ AI ตรวจสอบ";
      })
      .catch(err=>{
        statusDiv.textContent = "❌ ไม่สามารถเปิดกล้องได้: "+err.message;
        statusDiv.className = "error";
      });

    // -------- Popup helper --------
    function showSuccessPopup(msg, username){
      const wrap = document.createElement('div');
      wrap.className = 'success-popup-overlay';
      wrap.innerHTML = `
        <div class="success-popup-content">
          <div style="font-size:60px">✅</div>
          <h2>ยืนยันตัวตนสำเร็จ</h2>
          <p style="white-space:pre-line">${msg}</p>
          <button class="popup-btn" onclick="location.href='/profile?username='+username">ดูโปรไฟล์</button>
        </div>`;
      document.body.appendChild(wrap);
      setTimeout(()=>wrap.classList.add('show'),100);
      // auto redirect after 3s
      setTimeout(()=>location.href='/profile?username='+username,3000);
    }

    // -------- Login --------
    async function login(){
      loginBtn.disabled = true;
      statusDiv.textContent = "🧠 AI กำลังวิเคราะห์ใบหน้าของคุณ...";
      statusDiv.className = "loading";
      try{
        const canvas = document.createElement('canvas');
        canvas.width  = video.videoWidth;
        canvas.height = video.videoHeight;
        const ctx = canvas.getContext('2d');
        ctx.scale(-1,1);
        ctx.drawImage(video,-canvas.width,0);
        const imageData = canvas.toDataURL('image/jpeg').split(',')[1];
        const res = await fetch('/face_login',{
          method:'POST',
          headers:{'Content-Type':'application/json'},
          body:JSON.stringify({image:imageData})
        });
        const result = await res.json();
        if(result.success && result.action==="show_popup"){
          showSuccessPopup(result.popup_message,result.username);
          statusDiv.textContent = `✅ ${result.message}`;
          statusDiv.className = "success";
        }else if(result.success){
          window.location.href = `/profile?username=${result.username}`;
        }else{
          statusDiv.textContent = `❌ ${result.message}`;
          statusDiv.className = "error";
        }
      }catch(error){
        statusDiv.textContent = "❌ เกิดข้อผิดพลาด: "+error.message;
        statusDiv.className = "error";
      }
      loginBtn.disabled = false;
    }
  </script>
</body>
</html>
'''

# ----------------------------------------------------------------------
# PROFILE_PAGE (unchanged)
# ----------------------------------------------------------------------
PROFILE_PAGE = '''
<!DOCTYPE html>
<html>
<head>
  <title>โปรไฟล์ผู้ใช้</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    *{margin:0;padding:0;box-sizing:border-box}
    body{
      font-family:'Segoe UI',Tahoma,Geneva,Verdana,sans-serif;
      background:linear-gradient(135deg,#1abc9c 0%,#16a085 100%);
      min-height:100vh;display:flex;align-items:center;justify-content:center;padding:20px
    }
    .container{
      background:rgba(255,255,255,.95);backdrop-filter:blur(10px);
      border-radius:20px;padding:40px;box-shadow:0 20px 40px rgba(0,0,0,.1);
      text-align:center;max-width:450px;width:100%
    }
    .back-btn{
      position:absolute;top:20px;left:20px;background:rgba(255,255,255,.2);
      color:white;border:none;padding:10px 15px;border-radius:10px;cursor:pointer;
      font-size:16px;backdrop-filter:blur(10px)
    }
    h2{color:#333;margin-bottom:20px;font-size:28px}
    .profile-pic{
      width:150px;height:150px;border-radius:50%;object-fit:cover;
      border:5px solid #1abc9c;margin-bottom:20px;box-shadow:0 5px 15px rgba(0,0,0,.1)
    }
    .info-item{text-align:left;margin-bottom:15px;font-size:18px;color:#555}
    .info-item strong{color:#333;display:block;margin-bottom:5px}
    .btn{
      display:block;width:100%;padding:15px 30px;margin-top:30px;
      background:linear-gradient(45deg,#e74c3c,#c0392b);color:white;text-decoration:none;
      border-radius:12px;font-size:18px;font-weight:500;transition:.3s;border:none;cursor:pointer
    }
    .btn:hover{transform:translateY(-2px);box-shadow:0 10px 25px rgba(231,76,60,.3)}
  </style>
</head>
<body>
  <button class="back-btn" onclick="window.location.href='/'">← กลับ</button>
  <div class="container">
    <h2>👤 โปรไฟล์ผู้ใช้</h2>
    {% if user_image %}
      <img src="data:image/jpeg;base64,{{ user_image }}" alt="User Profile" class="profile-pic">
    {% else %}
      <p>ไม่พบรูปโปรไฟล์</p>
    {% endif %}
    <div class="info-item"><strong>ชื่อผู้ใช้ (Username):</strong> {{ user.username }}</div>
    <div class="info-item"><strong>ชื่อ-นามสกุลจริง:</strong> {{ user.fullname }}</div>
    <div class="info-item"><strong>เบอร์โทรศัพท์:</strong> {{ user.phone }}</div>
    <a href="/" class="btn">ออกจากระบบ</a>
  </div>
</body>
</html>
'''

# ----------------------------------------------------------------------
# SIGNUP_PAGE (คงเดิม - ยาว จึงไม่แก้รหัสในส่วนนี้)
# ----------------------------------------------------------------------
# ... (SIGNUP_PAGE string remains exactly asในโค้ดคุณ) ...
# เพื่อลดความยาวในตัวอย่าง ขอให้ใช้ SIGNUP_PAGE ตัวเดิมที่คุณมีได้เลย
# ----------------------------------------------------------------------

# ======================================================================
#                       BACK-END (DeepFace helpers)
# ======================================================================

# --- ฟังก์ชันใหม่: ปรับให้ยืดหยุ่น (flex mode) ---
def train_user_face_deepface(username, fullname, phone):
    """
    ใช้ DeepFace.represent กับ enforce_detection=False
    ต้องมีอย่างน้อย 3 รูปที่ตรวจเจอใบหน้า
    """
    user_dir = os.path.join(SAVE_DIR, username)
    embeddings_file = os.path.join(ENCODINGS_DIR, f"{username}_embeddings.pkl")

    if not os.path.exists(user_dir) or not os.listdir(user_dir):
        return False, "ไม่พบไดเร็กทอรีภาพของผู้ใช้หรือไม่มีภาพ", None

    user_embeddings, successful_images, first_image_saved_path = [], 0, None

    for i in range(1, 16):
        image_path = os.path.join(user_dir, f"angle_{i:02}.jpg")
        if not os.path.exists(image_path):
            continue
        try:
            reps = DeepFace.represent(img_path=image_path, model_name="VGG-Face",
                                      enforce_detection=False)
            if reps:
                user_embeddings.append(reps[0]["embedding"])
                successful_images += 1
                if first_image_saved_path is None:
                    first_image_saved_path = image_path
        except Exception as e:
            print(f"❌ {image_path}: {e}")

    if successful_images < 3:
        import shutil; shutil.rmtree(user_dir, ignore_errors=True)
        return False, f"พบใบหน้าได้เพียง {successful_images} รูป (ต้องอย่างน้อย 3)", None

    # ----- save pkl -----
    try:
        with open(embeddings_file, "wb") as f:
            pickle.dump({
                "username": username,
                "fullname": fullname,
                "phone": phone,
                "embeddings": user_embeddings,
                "num_images": successful_images,
                "first_image_path": first_image_saved_path
            }, f)
    except Exception as e:
        import shutil; shutil.rmtree(user_dir, ignore_errors=True)
        return False, f"ไม่สามารถบันทึก embeddings ได้: {e}", None

    # ----- update CSV -----
    try:
        df = pd.read_csv(USER_DATA_CSV)
        if username in df["username"].values:
            df.loc[df["username"] == username, ["fullname", "phone", "first_image_path"]] = \
                [fullname, phone, first_image_saved_path]
        else:
            df = pd.concat([df, pd.DataFrame([{
                "username": username,
                "fullname": fullname,
                "phone": phone,
                "first_image_path": first_image_saved_path
            }])], ignore_index=True)
        df.to_csv(USER_DATA_CSV, index=False, encoding="utf-8")
    except Exception as e:
        print(f"❌ CSV error: {e}")
        return True, f"เทรนสำเร็จ แต่บันทึก CSV ไม่สำเร็จ: {e}", username

    return True, f"✅ เทรน AI สำเร็จ! ใช้รูป {successful_images} รูป", username

# --- จดจำใบหน้า ---
def recognize_face_deepface(image_data):
    temp_image = "temp_login_image.jpg"
    try:
        Image.open(io.BytesIO(base64.b64decode(image_data))).save(temp_image)
        reps = DeepFace.represent(img_path=temp_image, model_name="VGG-Face",
                                  enforce_detection=False)
        if not reps:
            return False, "ไม่พบใบหน้าในภาพ", None, None, None
        unknown_emb = reps[0]["embedding"]

        best_user, best_dist, matched_data = None, float("inf"), None
        for file in os.listdir(ENCODINGS_DIR):
            if not file.endswith("_embeddings.pkl"): continue
            usr = file.replace("_embeddings.pkl", "")
            data = pickle.load(open(os.path.join(ENCODINGS_DIR, file), "rb"))
            for emb in data["embeddings"]:
                dist = np.linalg.norm(np.array(unknown_emb) - np.array(emb))
                if dist < best_dist:
                    best_dist, best_user, matched_data = dist, usr, data

        TH = 0.65
        if best_user and best_dist < TH:
            # confidence mapping
            conf = 70
            if best_dist <= .3:   conf = 95 + (0.3 - best_dist) * 16.67
            elif best_dist <= .45:conf = 85 + (0.45 - best_dist) * 66.67
            else:                 conf = 70 + (0.65 - best_dist) * 75
            conf = min(100, max(70, conf))
            fullname = matched_data.get("fullname", best_user)
            popup_msg = f"ยืนยันตัวตนสำเร็จ!\nยินดีต้อนรับคุณ {fullname}\nความมั่นใจ: {conf:.2f}%"
            return True, f"ยินดีต้อนรับคุณ {fullname}", best_user, matched_data, popup_msg
        else:
            return False, "ใบหน้าไม่ตรงกับข้อมูลในระบบ กรุณาลองใหม่", None, None, None
    except Exception as e:
        return False, f"เกิดข้อผิดพลาด: {e}", None, None, None
    finally:
        if os.path.exists(temp_image): os.remove(temp_image)

# ======================================================================
#                                ROUTES
# ======================================================================

@app.route("/")
def index():
    return render_template_string(HOME_PAGE)

@app.route("/login")
def login_page():
    return render_template_string(LOGIN_PAGE)

@app.route("/signup")
def signup_page():
    return render_template_string(SIGNUP_PAGE)  # ใช้ TEMPLATE เดิมของคุณ

# -------------------------
# SAVE IMAGE (ระหว่างสมัคร)
# -------------------------
@app.route("/save_image", methods=["POST"])
def save_image():
    data = request.get_json()
    username, angle, img_data = data.get("username"), data.get("angle"), data.get("image")
    if not username or not angle or not img_data:
        return jsonify(success=False, message="ข้อมูลไม่ครบ"), 400
    path = os.path.join(SAVE_DIR, username)
    os.makedirs(path, exist_ok=True)
    try:
        with open(os.path.join(path, f"angle_{int(angle):02}.jpg"), "wb") as f:
            f.write(base64.b64decode(img_data))
        return jsonify(success=True, message=f"📷 มุมที่ {angle} บันทึกเรียบร้อย!")
    except Exception as e:
        return jsonify(success=False, message=f"บันทึกรูปไม่สำเร็จ: {e}"), 500

# -------------------------
# TRAIN USER
# -------------------------
@app.route("/train_user", methods=["POST"])
def train_user():
    data = request.get_json()
    username, fullname, phone = data.get("username"), data.get("fullname"), data.get("phone")
    if not username or not fullname or not phone:
        return jsonify(success=False, message="ข้อมูลไม่ครบ"), 400
    # duplicate checks
    try:
        df = pd.read_csv(USER_DATA_CSV)
        if username in df["username"].values:
            return jsonify(success=False, message=f'ชื่อ "{username}" ใช้ไปแล้ว'), 400
        if phone in df["phone"].values:
            return jsonify(success=False, message=f'เบอร์ "{phone}" ใช้ไปแล้ว'), 400
    except Exception:
        pass
    ok, msg, usr = train_user_face_deepface(username, fullname, phone)
    if ok:
        return jsonify(success=True, message=msg, username=usr)
    return jsonify(success=False, message=msg), 400

# -------------------------
# FACE LOGIN  (🛠 FIXED)
# -------------------------
@app.route("/face_login", methods=["POST"])
def face_login():
    data = request.get_json()
    img_data = data.get("image")
    succ, msg, usr, udata, pop = recognize_face_deepface(img_data)
    if succ:
        # ดึง % จากข้อความ popup
        conf_str = pop.split("ความมั่นใจ:")[-1].strip() if pop else ""
        return jsonify(success=True, message=msg, username=usr,
                       popup_message=pop, confidence=conf_str,
                       action="show_popup")
    return jsonify(success=False, message=msg)

# -------------------------
# PROFILE PAGE (🛠 FIXED)
# -------------------------
@app.route("/profile")
def profile_page():
    username = request.args.get("username")
    if not username:
        return redirect(url_for("index"))
    try:
        df = pd.read_csv(USER_DATA_CSV)
        row = df[df["username"] == username]
        if row.empty:
            return "ไม่พบข้อมูลผู้ใช้", 404
        user = row.iloc[0].to_dict()
        img64 = ""
        if user.get("first_image_path") and os.path.exists(user["first_image_path"]):
            img64 = base64.b64encode(open(user["first_image_path"], "rb").read()).decode()
        return render_template_string(PROFILE_PAGE, user=user, user_image=img64)
    except Exception as e:
        return f"เกิดข้อผิดพลาด: {e}", 500

# ======================================================================
#                            RUN SERVER
# ======================================================================
if __name__ == "__main__":
    print("🚀 เริ่มต้นระบบ DeepFace (Flex Mode)…")
    try:
        ngrok.set_auth_token("2z7eJZzKAiolWcx2e88ntVbycoD_89oBXCN68jMmoif1huSVE")  # ← เปลี่ยนเป็น token จริง
        url = ngrok.connect(5000)
        print(f"🔗 Public URL: {url}")
        app.run(port=5000, use_reloader=False)
    except Exception as e:
        print(f"❌ Start-up error: {e}")


✅ DeepFace พร้อมใช้งาน
🚀 เริ่มต้นระบบ DeepFace (Flex Mode)…
🔗 Public URL: NgrokTunnel: "https://6a845d38324a.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:54:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:54:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:54:56] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:54:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:55:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:55:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:55:01] "GET /signup HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 16:55:18] "GET /?fbclid=PAQ0xDSwMBucRleHRuA2FlbQIxMAABpy1CjqbdisSBrw6Ff7qX8RZZEXY1VGDGZNNb2w4OqfnjicxUWwpUMUiJtJik_aem_aXTZsb_KcECLK9axAS1bNg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 